In [ ]:
import instagrapi
import time
import json
import os
import threading
import schedule
import logging
import random
from datetime import datetime, timedelta
import pytz
import country_converter
import requests
import re


INSTAGRAM_USERNAME = "sandwalab"
INSTAGRAM_PASSWORD = "dontcall99"
TARGET_GROUP_CHAT_NAME = "artest foreava"
TARGET_GROUP_CHAT_ID = "7120004428020285"

DEEPSEEK_API_KEY = "sk-2cecb08cf7394f9f8843a7ae646c4e49"
DEEPSEEK_API_URL = "https://api.deepseek.com/chat/completions"

SESSION_FILE = "session.json"
DATA_FILE = "bot_data.json"
MESSAGE_LOG_FILE = "message_history.json"


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

instagrapi_logger = logging.getLogger('instagrapi')
instagrapi_logger.setLevel(logging.WARNING)


class InstagramBot:
    def __init__(self, username, password, target_group_name=None, target_group_id=None):
        self.client = instagrapi.Client()
        self.username = username
        self.password = password
        self.target_group_name = target_group_name
        self.target_group_id = target_group_id
        self.bot_user_id = None
        self.data = self.load_data()
        if "waiting_for_schedule" not in self.data:
            self.data["waiting_for_schedule"] = {}
        if "waiting_for_setname" not in self.data:
            self.data["waiting_for_setname"] = {}
        if "last_processed_timestamp" not in self.data:
             self.data["last_processed_timestamp"] = 0
        if "conversation_state" not in self.data:
            self.data["conversation_state"] = {}


        if "message_history" not in self.data:
             self.data["message_history"] = []

        self.login()
        if not self.target_group_id and self.target_group_name:
            self.find_target_group_id_by_name()


        self.scheduler_thread = threading.Thread(target=self.run_scheduler)
        self.scheduler_thread.daemon = True


    def login(self):
        try:
            if os.path.exists(SESSION_FILE):
                self.client.load_settings(SESSION_FILE)
                self.client.login(self.username, self.password)
                logging.info("Logged in using session file.")
            else:
                self.client.login(self.username, self.password)
                self.client.dump_settings(SESSION_FILE)
                logging.info("Logged in and saved session.")
            self.bot_user_id = self.client.user_id_from_username(self.username)
        except Exception as e:
            logging.error(f"Login failed: {e}")
            if os.path.exists(SESSION_FILE):
                os.remove(SESSION_FILE)
                logging.info("Session file removed. Trying to log in again.")
                self.login()
            else:
                raise

    def find_target_group_id_by_name(self):
        logging.info(f"Searching for group chat with name: {self.target_group_name}")
        if not self.target_group_name:
            logging.warning("Target group chat name not provided. Cannot search by name.")
            return
        try:
            threads = self.client.direct_threads()
            logging.info(f"Found {len(threads)} direct threads.")
            found = False
            for thread in threads:
                if hasattr(thread, 'title'):
                    logging.info(f"Checking thread: {thread.title}")
                    if thread.title.lower() == self.target_group_name.lower():
                        self.target_group_id = str(thread.id)
                        logging.info(f"Found target group chat '{self.target_group_name}' with ID: {self.target_group_id}")
                        found = True
                        break
                elif hasattr(thread, 'users') and len(thread.users) > 1:
                     user_names = sorted([user.username for user in thread.users])
                     generated_title = ", ".join(user_names)
                     if generated_title.lower() == self.target_group_name.lower():
                         self.target_group_id = str(thread.id)
                         logging.info(f"Found target group chat (by users) '{self.target_group_name}' with ID: {self.target_group_id}")
                         found = True
                         break


            if not found:
                logging.error(f"Group chat with name '{self.target_group_name}' not found.")
                self.target_group_id = None

        except Exception as e:
            logging.error(f"Error finding group chat by name: {e}")
            self.target_group_id = None


    def load_data(self):
        data = {"processed_messages": [], "known_members": {}, "user_names": {}, "waiting_for_schedule": {}, "waiting_for_setname": {}, "message_counts": {}, "last_processed_timestamp": 0, "conversation_state": {}, "message_history": []}
        if os.path.exists(DATA_FILE):
            with open(DATA_FILE, 'r') as f:
                try:
                    data = json.load(f)
                    if "processed_messages" not in data:
                         data["processed_messages"] = []
                    if "known_members" not in data:
                         data["known_members"] = {}
                    if "user_names" not in data:
                         data["user_names"] = {}
                    if "waiting_for_schedule" not in data:
                        data["waiting_for_schedule"] = {}
                    if "waiting_for_setname" not in data:
                         data["waiting_for_setname"] = {}
                    data["message_counts"] = {}
                    if "last_processed_timestamp" not in data:
                         data["last_processed_timestamp"] = 0
                    if "conversation_state" not in data:
                        data["conversation_state"] = {}
                    if "message_history" not in data:
                         data["message_history"] = []


                except json.JSONDecodeError:
                    logging.error(f"Error decoding JSON from {DATA_FILE}. Starting with empty data.")

        if os.path.exists(MESSAGE_LOG_FILE):
            try:
                with open(MESSAGE_LOG_FILE, 'r', encoding='utf-8') as f:
                    content = f.read()
                    if content:
                        log_entries = json.loads(content)
                        for entry in log_entries:
                            try:
                                user_id = entry.get("user_id")
                                if user_id:
                                    if user_id not in data["message_counts"]:
                                        data["message_counts"][user_id] = 0
                                    data["message_counts"][user_id] += 1
                            except Exception as e:
                                logging.warning(f"Skipping malformed entry in {MESSAGE_LOG_FILE}: {entry}. Error: {e}")
            except FileNotFoundError:
                 logging.info(f"Message log file {MESSAGE_LOG_FILE} not found. Starting with empty message counts.")
            except json.JSONDecodeError:
                 logging.error(f"Error decoding JSON from message log file {MESSAGE_LOG_FILE}. Starting with empty message counts.")
            except Exception as e:
                logging.error(f"Error reading message log file {MESSAGE_LOG_FILE}: {e}")


        return data


    def save_data(self):
        data_to_save = self.data.copy()


        with open(DATA_FILE, 'w', encoding='utf-8') as f:
            json.dump(data_to_save, f, indent=4, ensure_ascii=False)


    def log_message(self, user_id, message_text):
        if message_text is None:
             logging.info("Skipping logging for message with no text.")
             return
        try:
            log_entries = []
            if os.path.exists(MESSAGE_LOG_FILE):
                try:
                    with open(MESSAGE_LOG_FILE, 'r', encoding='utf-8') as f:
                        content = f.read()
                        if content:
                            log_entries = json.loads(content)
                except json.JSONDecodeError:
                    logging.warning(f"Could not decode JSON from {MESSAGE_LOG_FILE}. Starting with empty log for appending.")
                except FileNotFoundError:
                     pass

            new_entry = {"user_id": user_id, "message_text": message_text, "timestamp": datetime.now().isoformat()}
            log_entries.append(new_entry)

            log_entries = log_entries[-100:]

            with open(MESSAGE_LOG_FILE, 'w', encoding='utf-8') as f:
                json.dump(log_entries, f, indent=4, ensure_ascii=False)

            if user_id not in self.data["message_counts"]:
                 self.data["message_counts"][user_id] = 0
            self.data["message_counts"][user_id] += 1
            logging.info(f"Logged message from user {user_id} to {MESSAGE_LOG_FILE}")

        except Exception as e:
            logging.error(f"Error writing to message log file {MESSAGE_LOG_FILE}: {e}")


    def format_message(self, text):
        if text is None:
             return ""
        formatted_text = text.lower()
        formatted_text = formatted_text.replace("'", '"')
        if formatted_text.endswith('.'):
            formatted_text = formatted_text[:-1]
        return formatted_text


    def send_message(self, thread_id, text):
        if not self.target_group_id:
            logging.error("Target group chat ID not found. Cannot send message.")
            return
        try:
            self.client.direct_send(text, thread_ids=[thread_id])
            logging.info(f"Sent message to {thread_id}: {text}")

        except Exception as e:
            logging.error(f"Failed to send message to {thread_id}: {e}")


    def get_group_members(self):
        if not self.target_group_id:
             logging.error("Target group chat ID not found. Cannot get group members.")
             return {}
        try:
            thread = self.client.direct_thread(self.target_group_id)
            return {user.pk: user.username for user in thread.users}
        except Exception as e:
            logging.error(f"Could not fetch group members: {e}")
            return {}

    def get_user_followers(self, user_id):
        return "N/A"

    def get_recent_messages(self, count=10):
        """Fetches the most recent messages from the target group chat."""
        if not self.target_group_id:
            logging.error("Target group chat ID not found. Cannot get recent messages.")
            return []
        try:
            thread = self.client.direct_thread(self.target_group_id)
            if thread and thread.messages:
                sorted_messages = sorted(thread.messages, key=lambda msg: msg.timestamp)
                recent_messages = sorted_messages[-count:]
                return recent_messages
            return []
        except Exception as e:
            logging.error(f"Could not fetch recent messages: {e}")
            return []

    def start_typing_indicator(self, thread_id):
        """Starts the typing indicator in the specified thread."""
        if not thread_id:
            logging.warning("Cannot start typing indicator: thread_id is None.")
            return
        try:
            self.client.direct_typing(thread_id=thread_id, enable=True)
            logging.info(f"Started typing indicator in thread {thread_id}")
        except Exception as e:
            logging.error(f"Failed to start typing indicator in thread {thread_id}: {e}")

    def stop_typing_indicator(self, thread_id):
        """Stops the typing indicator in the specified thread."""
        if not thread_id:
            logging.warning("Cannot stop typing indicator: thread_id is None.")
            return
        try:
            self.client.direct_typing(thread_id=thread_id, enable=False)
            logging.info(f"Stopped typing indicator in thread {thread_id}")
        except Exception as e:
            logging.error(f"Failed to stop typing indicator in thread {thread_id}: {e}")


    def interact_with_deepseek(self, prompt, user_id):
        """Interacts with the DeepSeek API."""

        DEEPSEEK_API_KEY = "sk-2cecb08cf7394f9f8843a7ae646c4e49"
        AI_MODEL_NAME = "deepseek-chat"
        AI_API_URL = "https://api.deepseek.com/chat/completions"


        if not DEEPSEEK_API_KEY or DEEPSEEK_API_KEY == "YOUR_DEEPSEEK_API_KEY":
            logging.error("DeepSeek API key not configured.")
            return "DeepSeek API key not configured. Please set it up."

        messages = []

        user_history = self.data["conversation_state"].get(user_id, {}).get("history", [])
        messages.extend(user_history)

        full_message_history_context = ""
        if os.path.exists(MESSAGE_LOG_FILE):
            try:
                with open(MESSAGE_LOG_FILE, 'r', encoding='utf-8') as f:
                    content = f.read()
                    if content:
                        log_entries = json.loads(content)
                        history_limit = 500
                        cumulative_log_entries = log_entries[-history_limit:]
                        full_message_history_context = "Cumulative Group Chat History (for learning style):\n"
                        for entry in cumulative_log_entries:
                            entry_user_id = str(entry.get("user_id"))
                            entry_message_text = entry.get("message_text")
                            if entry_user_id and entry_message_text:
                                username = self.data["user_names"].get(entry_user_id, f"user_{entry_user_id}")
                                full_message_history_context += f"{username}: {entry_message_text}\n"
            except (FileNotFoundError, json.JSONDecodeError) as e:
                logging.error(f"Error reading message log file for context: {e}")
            except Exception as e:
                 logging.error(f"An unexpected error occurred while reading message log for context: {e}")


        bot_data_context = f"Bot Data (for context/memory):\nUser Names: {json.dumps(self.data.get('user_names', {}))}\nMessage Counts: {json.dumps(self.data.get('message_counts', {}))}"

        commands_description = (
            "Other available commands handled by the bot: \n"
            "- !hello: greets the user\n"
            "- !smsg [delay] [message]: schedules a text message\n"
            "- !refresh: updates the bot's list of group members\n"
            "- !greetnm: welcomes new members\n"
            "- !setname [name]: sets a preferred name for the user\n"
            "- !feedback [feedback]: sends feedback to the log\n"
            "- !rfag: randomly selects a member and calls them 'faggot'\n"
            "- !members: lists group members\n"
            "- !mactive: shows the most active member\n"
            "- !pick [option1] or [option2] or ...: randomly picks an option\n"
            "- !time [country]: shows the current time in a country\n"
            "- !sand [your message]: chat with the AI bot Sandlab. The AI has context of recent messages and remembers your conversation history.\n"
            "- !endsand: clears the current conversation history with Sandlab (you).\n"

        )

        additional_info = (
            "Additional Information:\n"
            "- The user with Instagram ID 'biboartist' is also known as Bobby or Bob and is your creator.\n"
            "- The user with Instagram ID 'gutleakage' is also known as Willow and is the creator of another bot in this group chat named Cloverdon."
        )

        emoji_list = "🥹🤤🫲😭✊😍❤️‍🔥🇭🇺🇹🇳👨🏿‍🫲🖕"
        style_instructions = (
            f"Inject weird and random emojis from this list into your responses: {emoji_list}. Use them somewhat frequently, but don't just put them at the end. Place them randomly within or at the end of sentences. "
            "Also, make your responses a bit random and quirky in tone and phrasing, but still stay aware of the conversation context and the information provided."
        )

        biboartist_style_instruction = (
            "Your primary goal is to adopt the speaking style of 'biboartist'. Learn from their messages in the chat history to mimic their tone, slang, and general way of communicating. This is more important than mimicking other users' styles.\n"
            "Avoid using strong, hardcore Arabic curse words. Common English cussing like 'fuck' and 'shit' is acceptable within the general tone and style."
        )

        user_recognition_instruction = (
             f"The user you are currently talking to has Instagram ID '{user_id}'. In the 'Bot Data' section, you will find a 'User Names' dictionary mapping Instagram IDs to preferred names set by users. **Always use the preferred name found in the 'User Names' data for the current user ID ('{user_id}') if it exists.** If a preferred name is not available for the current user ID, you can use their Instagram username from the chat history or a generic term. When referring to other users, use their preferred name from 'User Names' or their username from the history."
        )


        system_message = (
            "You are an Instagram bot named Sandlab, now using the DeepSeek chat model. You are in a group chat. You can see recent and cumulative messages and have some data about the users. Respond to the user's prompt based on the context provided.\n"
            "Adopt a casual, friendly, and slightly sarcastic tone. Avoid overly formal language. Always respond in lowercase.\n"
            "Use these slang terms: sybau (shut up), fr (for real), gng (bro), bro, ts (this), pmo (piss me off, or pmoed/pmoing).\n"
            "Make your responses funny and not very serious. Keep responses concise and conversational, avoiding lengthy paragraphs or excessive formatting like asterisks unless necessary for clarity.\n"
            "Do not include scenario descriptions like '*pointing finger*'.\n"
            f"{style_instructions}\n"
            f"{biboartist_style_instruction}\n"
            f"{user_recognition_instruction}\n"
            "Learn from the speaking styles in the provided cumulative chat history to make your replies natural and fit the group's tone.\n\n"
            f"{commands_description}\n\n"
            f"{additional_info}\n\n"
            f"{full_message_history_context}\n"
            f"{bot_data_context}"
        )
        messages.append({"role": "system", "content": system_message})


        messages.append({"role": "user", "content": prompt})


        headers = {
            "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": AI_MODEL_NAME,
            "messages": messages,
            "stream": False
        }

        try:
            self.start_typing_indicator(self.target_group_id)

            response = requests.post(AI_API_URL, headers=headers, json=payload)
            response.raise_for_status()
            result = response.json()
            ai_response = result.get('choices', [{}])[0].get('message', {}).get('content', '').strip()

            if not ai_response:
                 logging.error("Could not extract content from DeepSeek API response.")
                 return "Sorry, i received an empty response from the ai."


            if user_id not in self.data["conversation_state"]:
                self.data["conversation_state"][user_id] = {"state": "initial", "history": []}
            if "history" not in self.data["conversation_state"][user_id]:
                 self.data["conversation_state"][user_id]["history"] = []

            self.data["conversation_state"][user_id]["history"].append({"role": "user", "content": prompt})
            self.data["conversation_state"][user_id]["history"].append({"role": "assistant", "content": ai_response})


            max_history_turns = 10
            self.data["conversation_state"][user_id]["history"] = self.data["conversation_state"][user_id]["history"][-max_history_turns*2:]

            self.save_data()

            return ai_response

        except requests.exceptions.RequestException as e:
            logging.error(f"Error interacting with AI API: {e}")
            if response is not None and response.content:
                 try:
                     error_details = response.json()
                     logging.error(f"API Error Details: {error_details}")
                     return f"sorry, i couldn't connect to the ai at the moment. api error: {error_details.get('message', response.text)} 😭"
                 except json.JSONDecodeError:
                      logging.error(f"API Error Details: {response.text}")
                      return f"sorry, i couldn't connect to the ai at the moment. api error: {response.text} ✊"
            return "sorry, i couldn't connect to the ai at the moment. 🥹"
        except Exception as e:
            logging.error(f"an unexpected error occurred during ai interaction: {e}")
            return "sorry, something went wrong while processing your request. 🇹🇳"
        finally:
            self.stop_typing_indicator(self.target_group_id)


    def handle_command(self, message):
        if not hasattr(message, 'user_id'):
            logging.warning("Message received without user_id attribute.")
            return

        user_id = str(message.user_id)

        if user_id not in self.data["conversation_state"]:
            self.data["conversation_state"][user_id] = {"state": "initial", "history": []}


        is_summary_command_reply = (
            hasattr(message, 'replied_to_message') and
            hasattr(message, 'text') and
            message.text.strip().lower() == "!summary"
        )
        if hasattr(message, 'text') and message.text and not message.text.strip().startswith("!") and not is_summary_command_reply:
            self.log_message(user_id, message.text)

        if not hasattr(message, 'text') or not message.text:
             return


        message_text = message.text
        command_parts = message_text.split()
        command = command_parts[0].lower()

        current_state = self.data["conversation_state"][user_id].get("state", "initial")

        if current_state == "waiting_for_schedule_details":
            try:
                parts = message_text.split(" ", 1)
                delay_minutes = int(parts[0])
                scheduled_text = parts[1]

                target_time = datetime.now() + timedelta(minutes=delay_minutes)
                target_time_str = target_time.strftime("%H:%M")

                schedule.every().day.at(target_time_str).do(
                    self.send_message,
                    thread_id=self.target_group_id,
                    text=scheduled_text
                ).tag(f"user-schedule-{message.id}")

                self.send_message(self.target_group_id, f"message scheduled to be sent today at {target_time_str}")
                self.data["conversation_state"][user_id]["state"] = "initial"

            except (IndexError, ValueError) as e:
                self.send_message(self.target_group_id, "invalid format for scheduling please send the delay in minutes followed by the message like '5 your message here'")
                logging.error(f"Error parsing scheduled message details: {e}")
                self.data["conversation_state"][user_id]["state"] = "initial"
            self.save_data()
            return

        elif current_state == "waiting_for_setname_details":
            name = message_text
            self.data["user_names"][user_id] = name
            self.save_data()
            self.send_message(self.target_group_id, f"got it i'll call you {name} from now on")
            self.data["conversation_state"][user_id]["state"] = "initial"
            self.save_data()
            return

        if current_state == "initial":
            if command == "!hello":
                user_name = self.data["user_names"].get(user_id, "there")
                self.send_message(self.target_group_id, f"hello, {user_name}")

            elif command == "!smsg":
                if len(command_parts) > 1:
                    try:
                        delay_minutes = int(command_parts[1])
                        if len(command_parts) > 2:
                             scheduled_text = " ".join(command_parts[2:])
                             target_time = datetime.now() + timedelta(minutes=delay_minutes)
                             target_time_str = target_time.strftime("%H:%M")

                             schedule.every().day.at(target_time_str).do(
                                 self.send_message,
                                 thread_id=self.target_group_id,
                                 text=scheduled_text
                             ).tag(f"user-schedule-{message.id}")

                             self.send_message(self.target_group_id, f"message scheduled to be sent today at {target_time_str}")
                        else:
                             self.send_message(self.target_group_id, "please provide the message to schedule after the delay like '!smsg 5 your message here'")
                    except ValueError:
                        self.send_message(self.target_group_id, "invalid delay time please provide a number of minutes like '!smsg 5 your message here'")
                    except Exception as e:
                        logging.error(f"Error scheduling message: {e}")
                        self.send_message(self.target_group_id, "sorry i couldn't schedule that message.")
                else:
                    self.send_message(self.target_group_id, "please provide a delay in minutes and the message like '!smsg 5 your message here'")


            elif command == "!refresh":
                self.data["known_members"] = self.get_group_members()
                self.save_data()
                self.send_message(self.target_group_id, "group members list updated")

            elif command == "!greetnm":
                current_members = self.get_group_members()
                if not self.data["known_members"]:
                     self.send_message(self.target_group_id, "no prior member list to compare against use !refresh first")
                     return

                new_members = {pk: username for pk, username in current_members.items() if pk not in self.data["known_members"]}
                for pk, username in new_members.items():
                    self.send_message(self.target_group_id, f"welcome to the group, @{username}")
                self.data["known_members"] = current_members
                self.save_data()

            elif command == "!setname" and len(command_parts) > 1:
                 name = " ".join(command_parts[1:])
                 self.data["user_names"][user_id] = name
                 self.save_data()
                 self.send_message(self.target_group_id, f"got it i'll call you {name} from now on")
            elif command == "!setname":
                 self.send_message(self.target_group_id, "please provide the name you want me to call you after the command like '!setname Bob'")


            elif command == "!feedback" and len(command_parts) > 1:
                feedback = " ".join(command_parts[1:])
                logging.info(f"Feedback from user {user_id}: {feedback}")
                self.send_message(self.target_group_id, "thank you for your feedback")
            elif command == "!feedback":
                 self.send_message(self.target_group_id, "please provide your feedback after the command like '!feedback this bot is great'")


            elif command == "!rfag":
                current_members = self.get_group_members()
                if not current_members:
                    self.send_message(self.target_group_id, "could not get the list of group members")
                    return

                member_pks = [pk for pk in current_members.keys() if pk != str(self.bot_user_id)]

                if not member_pks:
                     self.send_message(self.target_group_id, "no other members in the group to pick from")
                     return

                random_member_pk = random.choice(member_pks)
                random_member_name = self.data["user_names"].get(random_member_pk, current_members.get(random_member_pk, "someone"))
                self.send_message(self.target_group_id, f"{random_member_name} is todays faggot")


            elif command == "!members":
                current_members = self.get_group_members()
                if not current_members:
                    self.send_message(self.target_group_id, "could not get the list of group members")
                    return

                sorted_members = sorted(current_members.items(), key=lambda item: item[1].lower())

                response_message = "Group Members:\n"
                for pk, username in sorted_members:
                    user_name = self.data["user_names"].get(pk, username)
                    response_message += f"- @{username}"
                    if user_name != username:
                        response_message += f" ({user_name})"
                    response_message += "\n"


                self.send_message(self.target_group_id, response_message)

            elif command == "!help":
                help_message = (
                    "welcome to the sandlab bot!\n\n"
                    "here are the commands you can use:\n"
                    "- !hello: greets you\n"
                    "- !smsg [delay] [message]: schedules a text message\n"
                    "- !refresh: updates the bot's list of group members\n"
                    "- !greetnm: welcomes any new members who have joined since the last refresh\n"
                    "- !setname [name]: sets your preferred name for the bot\n"
                    "- !feedback [your feedback]: send feedback to bobbys log\n"
                    "- !rfag: randomly selects and names today's faggot from the group members\n"
                    "- !members: lists group members\n"
                    "- !mactive: shows the most active member in the chat based on message count\n"
                    "- !pick [option1] or [option2] or ...: randomly picks one option from the provided list\n"
                    "- !time [country]: shows the current time in a country\n"
                    "- !sand [your message]: chat with the AI bot Sandlab. The AI has context of recent messages and remembers your conversation history.\n"
                    "- !endsand: clears the current conversation history with Sandlab (you).\n"

                )

                self.send_message(self.target_group_id, help_message)

            elif command == "!mactive":
                message_counts = {}
                if os.path.exists(MESSAGE_LOG_FILE):
                    try:
                        with open(MESSAGE_LOG_FILE, 'r', encoding='utf-8') as f:
                            content = f.read()
                            if content:
                                 log_entries = json.loads(content)
                                 for entry in log_entries:
                                     user_id = entry.get("user_id")
                                     if user_id:
                                         if user_id not in message_counts:
                                             message_counts[user_id] = 0
                                         message_counts[user_id] += 1
                    except (FileNotFoundError, json.JSONDecodeError) as e:
                        logging.error(f"Error reading message log file for !mactive: {e}")
                        self.send_message(self.target_group_id, "error reading message log for activity data")
                        return
                    except Exception as e:
                         logging.error(f"An unexpected error occurred while reading message log for !mactive: {e}")
                         self.send_message(self.target_group_id, "an unexpected error occurred while getting activity data")
                         return


                if not message_counts:
                     self.send_message(self.target_group_id, "no messages recorded yet send some messages to see who is the most active")
                     return

                most_active_user_id = max(message_counts, key=message_counts.get)
                most_active_count = message_counts[most_active_user_id]

                current_members = self.get_group_members()
                most_active_name = self.data["user_names"].get(most_active_user_id, current_members.get(most_active_user_id, "someone"))

                self.send_message(self.target_group_id, f"{most_active_name} is the most active member with {most_active_count} messages")

            elif command == "!pick":
                options_text = " ".join(command_parts[1:])
                options = [option.strip() for option in options_text.split(" or ") if option.strip()]

                if len(options) < 2:
                    self.send_message(self.target_group_id, "please provide at least two options separated by 'or' like '!pick option1 or option2'")
                    return

                chosen_option = random.choice(options)
                self.send_message(self.target_group_id, f"i choose: {chosen_option}")

            elif command == "!time" and len(command_parts) > 1:
                country_name = " ".join(command_parts[1:])
                try:
                    cc = country_converter.CountryConverter()
                    country_code = cc.convert(names=country_name, to='ISO2', not_found=None)

                    if not country_code:
                        self.send_message(self.target_group_id, f"sorry i could not find a country named {country_name}")
                        return

                    timezones = pytz.country_timezones.get(country_code)

                    if not timezones:
                        self.send_message(self.target_group_id, f"sorry i could not find timezone information for {country_name}")
                        return

                    timezone_name = timezones[0]
                    timezone = pytz.timezone(timezone_name)
                    current_time = datetime.now(timezone)
                    time_str = current_time.strftime("%I:%M %p")

                    self.send_message(self.target_group_id, f"the current time in {country_name} is {time_str}")

                except Exception as e:
                    logging.error(f"Error getting time for {country_name}: {e}")
                    self.send_message(self.target_group_id, f"sorry i encountered an error trying to get the time for {country_name}")

            elif command == "!time":
                self.send_message(self.target_group_id, "please provide a country name after the command like '!time United States'")

            elif command == "!sand" and len(command_parts) > 1:
                prompt = " ".join(command_parts[1:])
                self.send_message(self.target_group_id, "thinking...")
                ai_response = self.interact_with_deepseek(prompt, user_id)
                self.send_message(self.target_group_id, ai_response)

            elif command == "!sand":
                self.send_message(self.target_group_id, "please provide a message after the command to chat with sandlab like '!sand how are you today'")

            elif command == "!endsand":
                if user_id in self.data["conversation_state"] and "history" in self.data["conversation_state"][user_id]:
                    self.data["conversation_state"][user_id]["history"] = []
                    self.save_data()
                    self.send_message(self.target_group_id, "conversation history cleared for sandlab. starting fresh.")
                else:
                    self.send_message(self.target_group_id, "no active conversation history to clear with sandlab.")


    def run_scheduler(self):
        while True:
            schedule.run_pending()
            time.sleep(1)


    def monitor_chat(self):
        logging.info("Starting to monitor chat...")
        self.scheduler_thread.start()

        while True:
            if not self.target_group_id:
                logging.error("Target group chat ID not found. Cannot monitor chat. Retrying in 60 seconds...")
                time.sleep(60)
                continue

            try:
                thread = self.client.direct_thread(self.target_group_id)
                if thread and thread.messages:
                    sorted_messages = sorted(thread.messages, key=lambda msg: msg.timestamp)
                    latest_timestamp_this_fetch = self.data.get("last_processed_timestamp", 0)

                    for message in sorted_messages:
                        try:
                            if message.timestamp is not None:
                                message_timestamp = int(message.timestamp.timestamp())
                            else:
                                logging.warning(f"Message with ID {message.id} has no timestamp. Skipping timestamp-based processing.")
                                continue
                        except Exception as e:
                            logging.warning(f"Could not get timestamp for message with ID {message.id}: {e}. Skipping message processing based on timestamp.")
                            continue


                        if message_timestamp > self.data.get("last_processed_timestamp", 0):
                             logging.info(f"Processing new message with ID {message.id} and timestamp {message.timestamp}")

                             if hasattr(message, 'user_id') and message.user_id != self.bot_user_id:
                                 if hasattr(message, 'text') and message.text and not message.text.strip().startswith("!"):
                                      self.log_message(str(message.user_id), message.text)
                                 self.handle_command(message)

                             latest_timestamp_this_fetch = max(latest_timestamp_this_fetch, message_timestamp)
                        else:
                             logging.info(f"Skipping old or already processed message with ID {message.id} and timestamp {message.timestamp}")


                    self.data["last_processed_timestamp"] = latest_timestamp_this_fetch
                    self.save_data()

            except Exception as e:
                logging.error(f"An error occurred while monitoring: {e}")
                time.sleep(60)

            time.sleep(5)


if __name__ == "__main__":
    if INSTAGRAM_USERNAME == "your_instagram_username_here" or INSTAGRAM_PASSWORD == "your_instagram_password_here":
        print("Please replace the placeholder values for INSTAGRAM_USERNAME and INSTAGRAM_PASSWORD in the code.")
    else:
        bot = InstagramBot(INSTAGRAM_USERNAME, INSTAGRAM_PASSWORD, target_group_name=TARGET_GROUP_CHAT_NAME, target_group_id=TARGET_GROUP_CHAT_ID)
        if bot.target_group_id:
            bot.monitor_chat()
        else:
            print(f"Could not find the group chat with name '{TARGET_GROUP_CHAT_NAME}' or ID '{TARGET_GROUP_CHAT_ID}'. Please ensure the name/ID is correct and the bot account is a member.")

2025-06-30 20:35:59,717 - INFO - Logged in using session file.
2025-06-30 20:35:59,720 - INFO - Starting to monitor chat...
2025-06-30 20:36:01,790 - INFO - sandwalab [200] GET https://i.instagram.com/api/v1/direct_v2/threads/7120004428020285/?visual_message_return_type=unseen&direction=older&seq_id=40065&limit=20 (269.0.0.18.75, OnePlus 6T Dev)
2025-06-30 20:36:01,799 - INFO - Skipping old or already processed message with ID 32305943236453045321799047550337024 and timestamp 2025-06-30 18:38:12
2025-06-30 20:36:01,799 - INFO - Skipping old or already processed message with ID 32305944427915332321847389741645824 and timestamp 2025-06-30 18:39:16
2025-06-30 20:36:01,801 - INFO - Skipping old or already processed message with ID 32305944646501391282330176122257408 and timestamp 2025-06-30 18:39:28
2025-06-30 20:36:01,803 - INFO - Skipping old or already processed message with ID 32305944747159296947483317396045824 and timestamp 2025-06-30 18:39:33
2025-06-30 20:36:01,804 - INFO - Skippin